In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import sys
sys.path.append('../../')
from scripts.viz_tools import animate_patch
from scripts.dl_utils import download_patch, rect_from_point

In [ ]:
# This function takes a while to run as it is extracting data from GEE

rect_width = 0.02
coords = [95.34799821290295, 5.606381719776017]
start_date = '2016-01-01'
end_date = '2021-11-01'
patches, raster_info = download_patch(rect_from_point(coords, rect_width), start_date, end_date)

In [ ]:
name = "pantai_beach"
cloud_threshold = .2
file_name = f"../../figures/videos/{name} {coords[1]:.2f}, {coords[0]:.2f} - thresh {cloud_threshold} - {start_date}-{end_date}.mp4"
animate_patch(patches, file_name, cloud_threshold=cloud_threshold, interval=100, size=8)

## Export Frames as Images

In [ ]:
from scripts import viz_tools
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

images = []
stretch = True
folder_name = file_name = f"../../figures/frames/{name} {coords[1]:.2f}, {coords[0]:.2f} - thresh {cloud_threshold} - {start_date}-{end_date}"
dates = [m['metadata']['']['id'][15:25] for m in raster_info]
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
for date, img in zip(dates, patches):
    if np.sum(img.mask) / np.size(img.mask) < cloud_threshold:
        rgb = viz_tools.normalize(img[:,:,3:0:-1])
        if stretch:
            rgb = viz_tools.stretch_histogram(rgb, 0.1, 1.0, gamma=1.2)
        im = Image.fromarray(np.uint8((np.clip(rgb, 0, 1) * 255)))
        im.save(f'{folder_name}/frame_{date}.png')